In [1]:
import os
%pwd

'f:\\ml_projects\\text_summarizer\\notbooks'

In [2]:
os.chdir('../')
%pwd

'f:\\ml_projects\\text_summarizer'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir:Path
    tokenizer_name:str
    data_path: Path

In [7]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILEPATH, params_filepath = PARAMS_FILEPATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        temp_config = self.config.data_transformation
        create_directories([temp_config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir = temp_config.root_dir,
            tokenizer_name = temp_config.tokenizer_name,
            data_path = temp_config.data_path 
        )
        return data_transformation_config

In [8]:
import os,sys
from text_summarizer.utils.logger import logger
from text_summarizer.utils.exception import CustomException
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

class DataTransformation():
    def __init__(self,config : DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self,examples_batch):
        input_encoding = self.tokenizer(examples_batch['dialogue'], max_length = 1024, truncation = True)

        with self.tokenizer.as_target_tokenizer():
            target_encoding = self.tokenizer(examples_batch['summary'], max_length = 128, truncation = True)

        return {
            'input_ids': input_encoding['input_ids'],
            'attention_mask': input_encoding['attention_mask'],
            'labels': target_encoding['input_ids'],
        }
    
    def convert(self):
        try:
            dataset_samsum = load_from_disk(self.config.data_path)
            dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features,batched = True)
            dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"dataset_samsum"))
            logger.info("Sucessfully Transformed Example into Features")
        except Exception as e:
            raise CustomException(e,sys)

In [9]:
import sys
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise CustomException(e,sys)

 [ 2024-01-09 12:42:38,990 ] - 25 - common - artifacts Directory Created Successfully
 [ 2024-01-09 12:42:38,993 ] - 25 - common - artifacts/data_transformation Directory Created Successfully


tokenizer_config.json: 100%|██████████| 88.0/88.0 [00:00<00:00, 17.6kB/s]
f:\ml_projects\text_summarizer\textsummarizer\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Lenovo\.cache\huggingface\hub\models--google--pegasus-cnn_dailymail. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
config.json: 10